In [1]:
from pyspark.sql import SparkSession
import math
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import boto3
import joblib
from botocore.config import Config
import tempfile
import logging
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sys
import subprocess

import os
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

In [2]:
ALL_MONTHS = ['202503']
fix_date = ['15']

## s3 config
s3_endpoint = 'http://access.icos.datalake.vnpt.vn:80'
s3_access_key = '47yFLHe5c6Vy9mJaF9fQ'
s3_secret_key =  'TZBxi4oQLmeo1nXyWkfwIGORM4RIuDSMaZntykuS'

## common config 
proxy = "http://10.144.13.144:3129"
bucket_name = "media_os_blueinfo_dev_2"
bucket_name_cos = f"cos://{bucket_name}.datalake"

## dir config
model_code = 'cs_lg_2025'
cs_data_dir = f"{bucket_name_cos}/scores/model_code={model_code}/merged_fts/date={ALL_MONTHS[0]}{fix_date[0]}"
lg_batch_dir = f'{bucket_name_cos}/scores/model_code={model_code}/lg_batch_predict/date={ALL_MONTHS[0]}{fix_date[0]}'
lg_output_path = f'{bucket_name_cos}/scores/model_code={model_code}/lg_predict_v2/date={ALL_MONTHS[0]}{fix_date[0]}'

In [3]:
def install_package():

    file_path = ["xgboost==2.1.1", "scipy==1.8.1", "optbinning==0.20.1", "catboost==1.2.7", "pandas==1.4.4"]

    cmd = [sys.executable, "-m", "pip", "install", "--no-cache-dir", "--proxy", proxy] + file_path

    try:
        result = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("Successfully install packages")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error when install packages", e)
        print(e.stdout)
        print(e.stderr)

In [4]:
install_package()

Successfully install packages
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/153.9 MB 2.0 MB/s eta 0:01:18
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/153.9 MB 3.4 MB/s eta 0:00:45
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/153.9 MB 7.0 MB/s eta 0:00:22
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/153.9 MB 15.5 MB/s eta 0:00:10
     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/153.9 MB 29.2 MB/s eta 0:00:06
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/153.9 MB 41.7 MB/s eta 0:00:04
     ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/153.9 MB 70.9 MB/s eta 0:00:03
     ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/153.9 MB 86.2 MB/s eta 0:00:02
     ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/153.9 MB 76.3 MB/s eta 0:00:02
     ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/153.9 MB 80.7 MB/s eta 0:00:02
     ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/153.9 MB 107.3 MB/s eta 0:00:02
     ━━━━

In [5]:
m1_features = ['f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_SMS_CNT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_SMS_ONNET_IC_CNT*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_SMS_CNT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_SMS_OFFNET_OG_CNT*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_SMS_CNT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_SMS_ONNET_IC_CNT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_VOICE_RVN_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_OG_USG*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_SMS_OFFNET_OG_CNT*s3_file_date*Filter=full_T-6-acc',
     'f4658_blueinfo_voice_msc_count_CALL_DURATION*s3_file_date*Filter=MOC_T-6',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_SMS_OFFNET_IC_CNT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_IC_USG*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_SMS_ONNET_IC_CNT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_IC_USG*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_min_VOICE_USG*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_IC_USG*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_SMS_CNT*s3_file_date*Filter=full_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_OG_USG*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_SMS_OFFNET_OG_CNT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_TOT_VOICE_RVN_AMT*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_SMS_ONNET_IC_CNT*s3_file_date*Filter=full_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_SMS_OFFNET_OG_CNT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_VOICE_RVN_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_SMS_CNT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_OG_USG*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_VOICE_ONNET_OG_USG*s3_file_date*Filter=full_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_VOICE_USG*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_SMS_OFFNET_OG_CNT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_SMS_OFFNET_OG_CNT*s3_file_date*Filter=full_T-6-acc',
     'f4548_blueinfo_voice_msc_sum_CALL_DURATION*s3_file_date*Filter=MOC_T-3']

m2_features =['f_blueinfo_smrs_dwd_geo_rvn_mtd_std_TOT_DATA_CHRGD_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_DATA_CHRGD_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_DATA_BYTES_SNT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_DATA_BYTES_RCVD*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_min_TOT_DATA_CHRGD_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_TOT_DATA_RVN_AMT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_TOT_DATA_CHRGD_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_DATA_USG*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_DATA_BYTES_RCVD*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_ocs_crs_usage_max_totalcost**Filter=CHARGINGCATEGORYNAME==DATA_T-5-acc',
     'f_blueinfo_ocs_crs_usage_std_totalcost**Filter=CHARGINGCATEGORYNAME==DATA_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_DATA_USG*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_DATA_BYTES_RCVD*s3_file_date*Filter=full_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_min_TOT_DATA_CHRGD_AMT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_DATA_RVN_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_DATA_USG*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_DATA_RVN_AMT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_DATA_BYTES_RCVD*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_DATA_USG*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_DATA_RVN_AMT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_ocs_crs_usage_std_totalcost**Filter=CHARGINGCATEGORYNAME==DATA_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_DATA_BYTES_RCVD*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_DATA_EVT_CNT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_DATA_BYTES_RCVD*s3_file_date*Filter=full_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_DATA_BYTES_SNT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_DATA_CHRGD_AMT*s3_file_date*Filter=full_T-6-acc',
     'f55_blueinfo_ggsn_sum_DURATION_MINUTES**Filter=full_T-4',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_DATA_RVN_AMT*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_ocs_crs_usage_count_totalcost**Filter=CHARGINGCATEGORYNAME==DATA_T-4-acc',
     'f_blueinfo_ocs_crs_usage_sum_totalcost**Filter=CHARGINGCATEGORYNAME==DATA_T-3-acc']

m3_features = ['f_blueinfo_vascdr_brandname_meta_unique_SENDER**Filter=finance_T-6-acc',
     'f4428_blueinfo_vascdr_utn_credit_log_max_CREDIT_AMOUNT**Filter=full_T-6',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_vascdr_brandname_meta_unique_SENDER**Filter=full_T-6-acc',
     'f4419_blueinfo_vascdr_utn_credit_log_avg_CREDIT_AMOUNT**Filter=full_T-5',
     'f3940_blueinfo_ccbs_ct_tra_std_TRATHUE**Filter=full_T-6',
     'f4422_blueinfo_vascdr_utn_credit_log_max_CREDIT_AMOUNT**Filter=full_T-5',
     'f4445_blueinfo_vascloud_da_max_value**Filter=full_T-2',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-5-acc',
     'f4425_blueinfo_vascdr_utn_credit_log_avg_CREDIT_AMOUNT**Filter=full_T-6',
     'f4451_blueinfo_vascloud_da_max_value**Filter=full_T-3',
     'f4459_blueinfo_vascloud_da_sum_value**Filter=full_T-5',
     'f4465_blueinfo_vascloud_da_sum_value**Filter=full_T-6',
     'f3929_blueinfo_ccbs_ct_tra_sum_TRAGOC**Filter=full_T-6',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-1-acc',
     'f3939_blueinfo_ccbs_ct_tra_max_TRATHUE**Filter=full_T-6',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_VAS_RVN_AMT*s3_file_date*Filter=full_T-3-acc',
    'f4187_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-1',
     'f4188_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-1',
     'f4194_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-2',
     'f4195_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-2',
     'f4201_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-3',
     'f4202_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-3',
     'f4208_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-4',
     'f4209_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-4',
     'f4215_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-5',
     'f4216_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-5',
     'f4222_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-6',
     'f4223_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-6',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_last_action**Filter=full_T-1-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_last_action**Filter=full_T-2-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_last_action**Filter=full_T-3-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_last_action**Filter=full_T-3-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_last_action**Filter=full_T-4-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_last_action**Filter=full_T-4-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_last_action**Filter=full_T-5-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_last_action**Filter=full_T-5-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_last_action**Filter=full_T-6-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_last_action**Filter=full_T-6-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_service_code**Filter=full_T-1-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_service_code**Filter=full_T-1-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_service_code**Filter=full_T-2-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_service_code**Filter=full_T-2-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_service_code**Filter=full_T-3-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_service_code**Filter=full_T-3-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_service_code**Filter=full_T-4-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_service_code**Filter=full_T-4-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_service_code**Filter=full_T-5-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_service_code**Filter=full_T-5-acc',
     'f_blueinfo_ccbs_spi_3g_subs_least-frequent-item_service_code**Filter=full_T-6-acc',
     'f_blueinfo_ccbs_spi_3g_subs_most-frequent-item_service_code**Filter=full_T-6-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=full_T-1-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=full_T-1-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=full_T-2-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=full_T-2-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=full_T-3-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=full_T-3-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=full_T-4-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=full_T-4-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=full_T-5-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=full_T-5-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=full_T-6-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=full_T-6-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=finance_T-1-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=finance_T-1-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=finance_T-2-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=finance_T-2-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=finance_T-3-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=finance_T-3-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=finance_T-4-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=finance_T-4-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=finance_T-5-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=finance_T-5-acc',
     'f_blueinfo_vascdr_brandname_meta_least-frequent-item_SENDER**Filter=finance_T-6-acc',
     'f_blueinfo_vascdr_brandname_meta_most-frequent-item_SENDER**Filter=finance_T-6-acc']

m4_features = ['f214_blueinfo_ocs_air_sum_reillamount**Filter=module==AdjustmentRecord_T-5',
     'f232_blueinfo_ocs_air_sum_reillamount**Filter=module==AdjustmentRecord_T-6',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_BLLD_AMT*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_min_TOT_RVN_AMT*s3_file_date*Filter=full_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_TOT_RVN_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_TOT_BLLD_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_RVN_AMT*s3_file_date*Filter=full_T-4-acc',
     'f183_blueinfo_ocs_air_std_reillamount**Filter=module==AdjustmentRecord_T-3',
     'f168_blueinfo_ocs_air_count_reillamount**Filter=full_T-3',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_RVN_AMT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_ratio_TOT_BLLD_AMT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_ELOAD_AMT*s3_file_date*Filter=full_T-2-acc',
     'f132_blueinfo_ocs_air_count_reillamount**Filter=full_T-1',
     'f178_blueinfo_ocs_air_sum_reillamount**Filter=module==AdjustmentRecord_T-3',
     'f208_blueinfo_ocs_air_sum_reillamount**Filter=module==RefillRecord_T-5',
     'f219_blueinfo_ocs_air_std_reillamount**Filter=module==AdjustmentRecord_T-5',
     'f_blueinfo_ocs_crs_usage_std_totalcost**Filter=CHARGINGCATEGORYNAME==SERVICE_CHARGING (SCAP)_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_avg_TOT_BLLD_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_ELOAD_AMT*s3_file_date*Filter=full_T-1-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_std_ELOAD_AMT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_BLLD_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_ELOAD_AMT*s3_file_date*Filter=full_T-2-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_RC_RVN_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_RVN_AMT*s3_file_date*Filter=full_T-5-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_count_TOT_RC_RVN_AMT*s3_file_date*Filter=full_T-6-acc',
     'f229_blueinfo_ocs_air_min_reillamount**Filter=module==RefillRecord_T-6',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_TOT_BLLD_AMT*s3_file_date*Filter=full_T-6-acc',
     'f_blueinfo_ocs_crs_usage_max_totalcost**Filter=CHARGINGCATEGORYNAME==SERVICE_CHARGING (SCAP)_T-4-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_max_ELOAD_AMT*s3_file_date*Filter=full_T-3-acc',
     'f_blueinfo_smrs_dwd_geo_rvn_mtd_sum_ELOAD_AMT*s3_file_date*Filter=full_T-1-acc']

m5_features = [
     'f4225_blueinfo_ccbs_cv207_most-frequent-value_ma_tinh**Filter=full_T-6',
     'f4218_blueinfo_ccbs_cv207_most-frequent-value_ma_tinh**Filter=full_T-5',
     'f4204_blueinfo_ccbs_cv207_most-frequent-value_ma_tinh**Filter=full_T-3',
     'f4211_blueinfo_ccbs_cv207_most-frequent-value_ma_tinh**Filter=full_T-4',
     'f4197_blueinfo_ccbs_cv207_most-frequent-value_ma_tinh**Filter=full_T-2',
     'f4190_blueinfo_ccbs_cv207_most-frequent-value_ma_tinh**Filter=full_T-1',
     'f4189_blueinfo_ccbs_cv207_unique_ma_tinh**Filter=full_T-1',
     'f4196_blueinfo_ccbs_cv207_unique_ma_tinh**Filter=full_T-2',
     'f4203_blueinfo_ccbs_cv207_unique_ma_tinh**Filter=full_T-3',
     'f4210_blueinfo_ccbs_cv207_unique_ma_tinh**Filter=full_T-4',
     'f4217_blueinfo_ccbs_cv207_unique_ma_tinh**Filter=full_T-5',
     'f4224_blueinfo_ccbs_cv207_unique_ma_tinh**Filter=full_T-6',
     'f4226_blueinfo_ccbs_cv207_least-frequent-value_ma_tinh**Filter=full_T-6',
     'f4205_blueinfo_ccbs_cv207_least-frequent-value_ma_tinh**Filter=full_T-3',
     'f4219_blueinfo_ccbs_cv207_least-frequent-value_ma_tinh**Filter=full_T-5',
     'f4212_blueinfo_ccbs_cv207_least-frequent-value_ma_tinh**Filter=full_T-4',
     'GSMA_Operating_System',
     'Standardised_Full_Name',
     'Standardised_Device_Vendor',
     'Standardised_Marketing_Name',
     'Total_Ram',
     'Age_group',
     'f4187_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-1',
     'f4188_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-1',
     'f4194_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-2',
     'f4195_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-2',
     'f4201_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-3',
     'f4202_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-3',
     'f4208_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-4',
     'f4209_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-4',
     'f4215_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-5',
     'f4216_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-5',
     'f4222_blueinfo_ccbs_cv207_least-frequent-item_ma_tinh**Filter=full_T-6',
     'f4223_blueinfo_ccbs_cv207_most-frequent-item_ma_tinh**Filter=full_T-6',
    'Year_Released'
    ]

In [6]:
def get_s3_client():
    s3 = boto3.client('s3',
                     endpoint_url = s3_endpoint,
                     aws_access_key_id = s3_access_key,
                     aws_secret_access_key = s3_secret_key,
                     config = Config(signature_version = 's3v4'))
    return s3

def load_pickle_file(key):
    bucket = bucket_name
    s3_client = get_s3_client()
    try:
        with tempfile.TemporaryFile() as fp:
            s3_client.download_fileobj(Fileobj=fp, Bucket = bucket, Key = key)
            fp.seek(0)
            return joblib.load(fp)
    except Exception as e:
        raise logging.exception(e)

        
def load_model_config(spark, path):
    
    df = spark.read.parquet(f"{bucket_name_cos}/models/{path}")
    
    return df

In [7]:
def create_spark_instance(run_mode = 'prod'):
    clear_checkpoint_dir(run_mode)
    spark = SparkSession.builder \
            .appName(f'SparkApp_{run_mode}') \
            .config("spark.jars", "/code/libs/singlestore-spark-connector_2.12-4.1.3-spark-3.3.0.jar,/code/libs/singlestore-jdbc-client-1.1.5.jar,/code/libs/commons-dbcp2-2.9.0.jar,/code/libs/commons-pool2-2.11.1.jar,/code/libs/spray-json_3-1.3.6.jar") \
            .config("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED") \
            .config("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED") \
            .config("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED") \
            .config("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED") \
            .getOrCreate()

    sc = spark.sparkContext
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set("fs.cos.datalake.endpoint", s3_endpoint)
    hconf.set("fs.cos.datalake.access.key", s3_access_key)
    hconf.set("fs.cos.datalake.secret.key", s3_secret_key)
    
    spark.conf.set("spark.sql.adaptive.enabled", "true")
    if run_mode == 'prod': 
        spark.sparkContext.setCheckpointDir(f"{bucket_name_cos}/tmp/prod_checkpoints")
    else: 
        spark.sparkContext.setCheckpointDir(f"{bucket_name_cos}/tmp/bt_checkpoints")
    sc.setLogLevel("WARN")

    return spark

def clear_checkpoint_dir(run_mode):
    
    cos_endpoint = s3_endpoint
    cos_access_key = s3_access_key
    cos_secret_key = s3_secret_key  
#     bucket_name = bucket_name
    if run_mode == 'prod':
        checkpoint_prefix = "tmp/prod_checkpoints/"
    else: 
        checkpoint_prefix = "tmp/bt_checkpoints/"

    s3 = boto3.client(
        "s3",
        endpoint_url=cos_endpoint,
        aws_access_key_id=cos_access_key,
        aws_secret_access_key=cos_secret_key
    )
    
    while True: 
        objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=checkpoint_prefix)

        if "Contents" in objects:
            delete_keys = [{"Key": obj["Key"]} for obj in objects["Contents"]]
            s3.delete_objects(Bucket=bucket_name, Delete={"Objects": delete_keys})
        else:
            print(f"Deleted checkpoint directory: {checkpoint_prefix}")
            break

In [8]:
def winsorize(data, threshold):
    # handle outlier
    result = data.copy()
    threshold_dict = threshold.set_index('feature').to_dict('index')
    for feature, thresholds in threshold_dict.items():
        if feature in result.columns:
            lower_bound = thresholds['lower_threshold'] 
            upper_bound = thresholds['higher_threshold']
        
            result[feature] = result[feature].clip(lower = lower_bound, upper = upper_bound)
    return result

def scoring(df):    
    # models

    m1 = load_pickle_file("models/m1/20241022_m1.parquet")
    m2 = load_pickle_file("models/m2/20241022_m2.parquet")
    m3 = load_pickle_file("models/m3/20241022_m3.parquet")
    m4 = load_pickle_file("models/m4/20241022_m4.parquet")
    m5 = load_pickle_file("models/m5/20241022_m5.parquet")
    m6 = load_pickle_file("models/m6/20241022_m6.parquet")
    
    # winsorize tables
    m1_winsorize = load_model_config(spark , "m1/20241022_winsorize_table.parquet").toPandas()
    m2_winsorize = load_model_config(spark , "m2/20241022_winsorize_table.parquet").toPandas()
    m3_winsorize = load_model_config(spark , "m3/20241022_winsorize_table.parquet").toPandas()
    m4_winsorize = load_model_config(spark , "m4/20241022_winsorize_table.parquet").toPandas()
    m5_winsorize = load_model_config(spark , "m5/20241022_winsorize_table.parquet").toPandas()
    
    
    component_models = [[m1_winsorize, m1_features, m1],
                       [m2_winsorize, m2_features, m2],
                       [m3_winsorize, m3_features, m3],
                       [m4_winsorize, m4_features, m4],
                       [m5_winsorize, m5_features, m5]]
    
    
    def get_score(df, component_models, m6):
                
        # Create a new column "batch_id" based on the last character of msisdn
        df = df.withColumn("batch_id", F.expr("substring(msisdn, length(msisdn)-2, 3)"))

        # Get the distinct batch_ids
        distinct_batch_ids = df.select("batch_id").distinct().collect()

        # Iterate over the distinct batch_ids and yield batches
        for batch_id_row in distinct_batch_ids:
            batch_id = batch_id_row["batch_id"]
            
            # Filter the DataFrame based on the current batch_id
            batch_df = df.where(f"batch_id = '{batch_id}'")
        
            i = 2

            # get score for m1
            winsorize_table = component_models[0][0]
            features = component_models[0][1]
            model = component_models[0][2]

            # get data
            df_fts = batch_df.select("msisdn", *features).toPandas()

            # winsorize
            m1 = winsorize(df_fts.copy(), winsorize_table)

            # get prediction
            model_id = m1[['msisdn']]
            model_score = m1.drop(columns = ['msisdn'])
            model_proba = pd.DataFrame(model.predict_proba(model_score)[:,1])
            output = pd.concat([model_id.reset_index(), model_proba], axis=1)
            output[0] = output[0].round(4)
            output[0] = output[0] * 10000
            output = output.rename(columns = {0: 'm1_score'})
            output = output.drop(columns =['index'])
            output['m1_score'] = output['m1_score'].round()


            for winsorize_table, features, model in component_models[1:]:
                # winsorize
                df_fts = batch_df.select("msisdn", *features).toPandas()
                data = winsorize(df_fts.copy(), winsorize_table)

                # get prediction
                model_id = data[['msisdn']]
                model_score = data.drop(columns = ['msisdn'])
                model_proba = pd.DataFrame(model.predict_proba(model_score)[:,1])
                output2 = pd.concat([model_id.reset_index(), model_proba], axis=1)
                output2[0] = output2[0].round(4)
                output2[0] = output2[0] * 10000
                output2 = output2.rename(columns = {0: f'm{i}_score'})
                output2 = output2.drop(columns =['index'])
                output2[f'm{i}_score'] = output2[f'm{i}_score'].round()

                # increment
                i = i + 1

                # join data
                output = output.merge(output2, on = ['msisdn'], how = 'inner')

            # get ensemble output
            model_id = output
            model_score = output.drop(columns = ['msisdn'])
            model_proba = pd.DataFrame(m6.predict_proba(model_score)[:,1])
            output = pd.concat([model_id.reset_index(), model_proba], axis=1)
            output[0] = output[0].round(4)
            output[0] = output[0] * 10000
            output = output.rename(columns = {0: 'm6_score'})
            output = output.drop(columns =['index'])
            output['m6_score'] = output['m6_score'].round()        
        
            # convert to spark dataframe
            return_spark_df = spark.createDataFrame(output)

            # save to parquet
            return_spark_df.write.mode('overwrite').parquet(f'{lg_batch_dir}/batch_id={batch_id}')
    get_score(df, component_models, m6)

In [9]:
### main 
spark = create_spark_instance()

Deleted checkpoint directory: tmp/prod_checkpoints/


In [11]:
############## load dataset
dataset = spark.read.parquet(cs_data_dir)

In [12]:
### rename column
not_exists = ['f4658_blueinfo_voice_msc_count_CALL_DURATION*s3_file_date*Filter=MOC_T-6',
 'f4548_blueinfo_voice_msc_sum_CALL_DURATION*s3_file_date*Filter=MOC_T-3',
 'f214_blueinfo_ocs_air_sum_reillamount**Filter=module==AdjustmentRecord_T-5',
 'f232_blueinfo_ocs_air_sum_reillamount**Filter=module==AdjustmentRecord_T-6',
 'f183_blueinfo_ocs_air_std_reillamount**Filter=module==AdjustmentRecord_T-3',
 'f168_blueinfo_ocs_air_count_reillamount**Filter=full_T-3',
 'f178_blueinfo_ocs_air_sum_reillamount**Filter=module==AdjustmentRecord_T-3',
 'f208_blueinfo_ocs_air_sum_reillamount**Filter=module==RefillRecord_T-5',
 'f219_blueinfo_ocs_air_std_reillamount**Filter=module==AdjustmentRecord_T-5',
 'f229_blueinfo_ocs_air_min_reillamount**Filter=module==RefillRecord_T-6']

not_exists_dct = {}
for col1 in not_exists: 
    old_col = col1.split("blueinfo")[1:]
    for col2 in dataset.columns: 
        new_col = col2.split("blueinfo")[1:]
        if old_col == new_col: 
            not_exists_dct[col2] = col1
            
for k, v in not_exists_dct.items(): 
    dataset = dataset.withColumnRenamed(k, v)
############

In [13]:
scoring(dataset)

/opt/ibm/conda/miniconda/lib/python/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 9, 15, 17] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/ibm/conda/miniconda/lib/python/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 9, 15] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/ibm/conda/miniconda/lib/python/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 15] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/ibm/conda/miniconda/lib/python/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [0, 1, 2, 15] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/ibm/conda/minicon

In [14]:
##### load full data and calculate lg_perct_y_new
df_joined = spark.read.parquet(f"{lg_batch_dir}")

df_joined_columns = df_joined.columns
subcriber = spark.read.parquet(f"{bucket_name_cos}/features_v2/sub/final_fts/date={ALL_MONTHS[0]}{fix_date[0]}/")
df_joined = df_joined.join(subcriber, "msisdn", "left")
sdf_joined = df_joined.select(*df_joined_columns)


win = Window.orderBy(F.asc("m6_score"))
df_joined = df_joined.withColumn('lg_perct', F.ntile(100).over(win)) # lg_perct_y_new
df_joined.write.mode('overwrite').parquet(lg_output_path)

In [15]:
1

1